# Phase 1: 数据诊断

## 目标

系统化诊断陌生数据的质量和特征，为后续建模提供决策依据。

## 诊断流程

```
数据规模 → 数据类型 → 缺失值 → 异常值 → 分布特征 → 相关性 → 质量评分
```

## 输出

- 数据诊断报告（JSON + HTML）
- 可视化图表（缺失值、异常值、分布、相关性）
- 数据质量评分
- 下一步处理建议

---

## 1. 环境准备

In [ ]:
# 基础库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 导入自定义模块
from src import data_diagnosis as dd
from src import visualization as viz
from config import get_default_config

# 配置
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')

config = get_default_config()
print("✅ 环境准备完成!")
print(f"配置模式: {config.MODE}")
print(f"输出目录: {config.PHASE1_OUTPUT}")

## 2. 加载数据

使用Titanic数据集（分类问题，包含缺失值、类别不平衡等典型问题）

In [ ]:
# 加载Titanic数据集
df = sns.load_dataset('titanic')
target_col = 'survived'

print(f"✅ 数据加载成功! 形状: {df.shape[0]} 行 × {df.shape[1]} 列")
print(f"目标变量: {target_col}")
df.head()

## 3. 数据规模检查

检查样本数、特征数，判断数据量级对算法选择的影响。

In [ ]:
# 基础信息
basic_info = dd.basic_info(df, show=True)

n_samples = basic_info['n_samples']
n_features = basic_info['n_features']

# 量级判断
print(f"\n数据量级: ", end="")
if n_samples < 1000:
    print(f"小数据集({n_samples}行) → 建议简单模型")
elif n_samples < 100000:
    print(f"中等数据集({n_samples}行) → 可用集成学习")
else:
    print(f"大数据集({n_samples}行) → 可用LightGBM/深度学习")

print(f"特征数量: {n_features} → ", end="")
if n_features / n_samples > 0.1:
    print("特征相对较多，考虑正则化/特征选择")
else:
    print("比例合理")

## 4. 数据类型识别

识别数值型、类别型、ID型、常量等，决定预处理策略。

In [ ]:
# 类型推断
column_types = dd.infer_column_types(df, categorical_threshold=20)
column_summary = dd.column_summary(df)

display(column_summary)

# 处理建议
if column_types['id']:
    print(f"⚠️  ID列: {column_types['id']} → 建模前移除")
if column_types['constant']:
    print(f"⚠️  常量列: {column_types['constant']} → 无信息，移除")
if column_types['categorical']:
    print(f"✓ 类别型({len(column_types['categorical'])}个) → 需编码")
if column_types['datetime']:
    print(f"✓ 时间型({len(column_types['datetime'])}个) → 提取特征")

## 5. 缺失值分析

检测缺失值数量、位置和模式（MCAR/MAR/MNAR），决定填充或删除策略。

In [ ]:
# 缺失值统计
missing_stats = dd.missing_value_analysis(df, threshold=0.0)

if len(missing_stats) > 0:
    # 缺失模式检测
    missing_patterns = dd.detect_missing_pattern(df)
    
    # 可视化
    viz.plot_missing_bar(df)
    viz.plot_missing_heatmap(df)
else:
    print("✅ 无缺失值")

## 6. 异常值检测

使用IQR和Z-Score方法检测异常值，评估对模型的影响。

In [ ]:
# 数值型特征异常值检测
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

if len(numeric_cols) > 0:
    # IQR方法
    outlier_report_iqr = dd.outlier_analysis(df, method='iqr', columns=numeric_cols)
    
    # Z-Score方法
    outlier_report_zscore = dd.outlier_analysis(df, method='zscore', columns=numeric_cols)
    
    # 可视化（前6个特征）
    viz.plot_boxplots(df, columns=numeric_cols[:6])
    
    print("\n处理建议: 线性模型需移除/截断，树模型可保留")
else:
    print("⚠️  无数值型特征")

## 7. 数据分布分析

分析偏度、峰度，判断是否需要变换（对数、Box-Cox等）。

In [ ]:
# 数值特征分布
if len(numeric_cols) > 0:
    dd.distribution_analysis(df, columns=numeric_cols[:6])
else:
    print("⚠️  无数值型特征")

# 类别特征分布
categorical_cols = column_types['categorical'] + column_types['binary']
if len(categorical_cols) > 0:
    sample_cat_col = categorical_cols[0]
    viz.plot_categorical_distribution(df, sample_cat_col, top_n=15)

## 8. 相关性分析

发现强相关特征对（多重共线性）和与目标变量相关的重要特征。

In [ ]:
# 相关性热力图
if len(numeric_cols) >= 2:
    corr_matrix = dd.correlation_analysis(df, method='pearson', threshold=0.5)
    
    # 与目标变量的相关性
    if target_col in df.columns and target_col in numeric_cols:
        print(f"\n与目标变量 '{target_col}' 的相关性（Top 10）:")
        target_corr = corr_matrix[target_col].drop(target_col).sort_values(key=abs, ascending=False)
        
        for feature, corr_val in target_corr.head(10).items():
            strength = "强" if abs(corr_val) > 0.7 else "中" if abs(corr_val) > 0.4 else "弱"
            print(f"  {feature:<20} {corr_val:>7.3f} ({strength})")
else:
    print("⚠️  特征不足，无法分析相关性")

## 9. 数据质量评分

综合评分（0-100），判断数据是否ready for modeling。

In [ ]:
# 质量评分
quality_score = dd.calculate_data_quality_score(df, target=target_col)
total_score = quality_score['total_score']

print(f"数据质量评分: {total_score:.1f}/100 ({quality_score['grade']})")

# 建议
if total_score >= 80:
    print("✅ 质量良好，可进入Phase 2")
elif total_score >= 70:
    print("⚠️  质量中等，建议处理主要问题后建模")
else:
    print("❌ 质量较差，需重点改善数据")

## 10. 综合摘要图

In [ ]:
viz.plot_data_quality_summary(df, target=target_col)

## 11. 生成诊断报告

保存完整诊断结果为JSON和HTML格式。

In [ ]:
# 生成报告
report_path = config.PHASE1_OUTPUT / 'diagnosis_report'

diagnosis_report = dd.generate_diagnosis_report(
    df, 
    target=target_col,
    save_path=str(report_path)
)

print(f"✅ 报告已生成:")
print(f"   JSON: {report_path}.json")
print(f"   HTML: {report_path}.html")

## 12. 诊断总结与行动清单

In [ ]:
print("=" * 60)
print("Phase 1 诊断总结".center(60))
print("=" * 60)

print(f"\n1. 数据规模: {basic_info['n_samples']:,} 行 × {basic_info['n_features']} 列")
print(f"2. 内存: {basic_info['memory_usage_mb']:.2f} MB")
print(f"3. 质量评分: {quality_score['total_score']:.1f}/100 ({quality_score['grade']})")
print(f"4. 缺失值: {df.isnull().sum().sum()} 个")
print(f"5. 重复行: {basic_info['duplicated_rows']} 行")

if target_col in df.columns:
    print(f"\n目标变量 '{target_col}':")
    if df[target_col].nunique() <= 20:
        print(f"  类型: 分类 ({df[target_col].nunique()} 类)")
        value_counts = df[target_col].value_counts()
        max_ratio = value_counts.max() / len(df)
        print(f"  类别平衡: 最大类占比 {max_ratio*100:.1f}%")
        if max_ratio > 0.8:
            print(f"  ⚠️  严重不平衡!")
    else:
        print(f"  类型: 回归")
        print(f"  范围: [{df[target_col].min():.2f}, {df[target_col].max():.2f}]")

print("\n" + "=" * 60)

### 进入Phase 2前的准备工作

In [ ]:
print("📋 Phase 2 准备清单:")
print("=" * 60)

todo_list = []

if df.isnull().sum().sum() > 0:
    todo_list.append("处理缺失值（填充/删除）")
if column_types['id']:
    todo_list.append(f"移除ID列: {', '.join(column_types['id'])}")
if column_types['constant']:
    todo_list.append(f"移除常量列: {', '.join(column_types['constant'])}")
if basic_info['duplicated_rows'] > 0:
    todo_list.append(f"移除{basic_info['duplicated_rows']}行重复数据")
if column_types['categorical']:
    todo_list.append(f"编码{len(column_types['categorical'])}个类别特征")
if len(numeric_cols) > 0:
    todo_list.append("决定异常值处理策略")

# 类别不平衡检查
if target_col in df.columns and df[target_col].nunique() <= 20:
    value_counts = df[target_col].value_counts()
    if value_counts.max() / len(df) > 0.7:
        todo_list.append("处理类别不平衡（SMOTE/权重调整）")

if todo_list:
    for i, task in enumerate(todo_list, 1):
        print(f"  {i}. {task}")
else:
    print("  ✅ 无需特殊处理，可直接建模")

print("\n" + "=" * 60)
print("\n✅ Phase 1 完成! 下一步: phase2_quick_baseline.ipynb")
print("=" * 60)

## 参考资料

- **工作流指南**: `../ML_WORKFLOW_GUIDE.md` - 数据诊断决策树
- **详细方法**: `../01_data_diagnosis_framework/data_problem_to_solution_mapping.md`
- **快速参考**: `../01_data_diagnosis_framework/data_diagnosis_quick_reference.md`

---

**下一步**: [Phase 2: 快速Baseline](phase2_quick_baseline.ipynb)